In [0]:
# Define o caminho apontando para dentro da subpasta SalesLT
caminho_gold = "abfss://gold@sabinoanalyticsdatalake.dfs.core.windows.net/"
caminho_silver = "abfss://silver@sabinoanalyticsdatalake.dfs.core.windows.net/"

# Lista os arquivos dentro dessa pasta específica
display(dbutils.fs.ls(caminho_silver + 'SalesLT'))


In [0]:
table_name = []

for i in dbutils.fs.ls(caminho_silver + 'SalesLT'):
    print(i.name.split('/')[0])
    table_name.append(i.name.split('/')[0])

In [0]:
table_name

In [0]:
from pyspark.sql.functions import from_utc_timestamp, date_format
from pyspark.sql.types import TimestampType

for name in table_name:
    path = caminho_silver + 'SalesLT/' + name
    df = spark.read.format('delta').load(path)
    column_names = df.columns

    for old_col_name in column_names:
        new_col_name = "".join("_" + char if char.isupper() and not old_col_name[i - 1].isupper() else char for i, char in enumerate(old_col_name)).lstrip("_")
        df = df.withColumnRenamed(old_col_name, new_col_name)
        
        
            
    output_path = caminho_gold + 'SalesLT/' + name + '/'
    df.write.mode('overwrite').format('delta').save(output_path)


In [0]:
display(df)

In [0]:
gold_directories = dbutils.fs.ls(caminho_gold + 'SalesLT')
for directory in gold_directories:
    directory_path = directory.path
    folder_name = directory.name
    table_name = folder_name.replace("/", "")
    df = spark.read.format('delta').load(directory_path)
    df.write.saveAsTable(table_name)

In [0]:
display(dbutils.fs.ls(caminho_gold + 'SalesLT'))